<a href="https://colab.research.google.com/github/MrEktirir/Antibiotics-Resistance-Prediction/blob/feature%2FExplotary_Data_Analysis/Notebooks/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

from google.colab import drive
drive.mount('/content/drive')

In [46]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
import time
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import cm
from tqdm import tqdm
from Bio.Seq import Seq

# Sklearn Module
from sklearn.model_selection import KFold,GroupKFold,GridSearchCV,StratifiedKFold
from sklearn.metrics import confusion_matrix,balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer,RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve,ShuffleSplit
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN

# Machine Learning Models
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
#from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBRegressor,XGBClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import os,warnings;warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Reading Files

In [6]:
azm_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/azm_sr_gwas_filtered_unitigs.Rtab', sep="\t")
cfx_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/cfx_sr_gwas_filtered_unitigs.Rtab', sep="\t")
cip_sr_df = pd.read_csv('/content/drive/MyDrive/Dataset/cip_sr_gwas_filtered_unitigs.Rtab', sep="\t")
metadata_df = pd.read_csv('/content/drive/MyDrive/Dataset/metadata.csv')

# Basic Stats

# **1. Introduction**

## 1.1 Background

RESISTANCE OF BACTERIA

  We will be focussing on a species called Neisseria gonorrhoeae, the bacteria which cause gonorrhoea.
Gonorrhoea is the second most common sexually transmitted infection (STI) in Europe, after chlamydia.
Rates of gonorrhoea infection are on the rise, with a 26% increase reported from 2017-2018 in the UK.

  Many people who are infected (especially women) experience no symptoms, helping the disease to spread.
If the infection is left untreated, it can lead to infertility in women, and can occasionally spread to
other parts of the body such as your joints, heart valves, brain or spinal cord.
Resistance of these bacteria to antibiotics is rising over time, making infections hard to treat.

* In the past, patients were treated with an antibiotic called ciprofloxaxcin.
* Doctors had to stop using this antibiotic because resistance to the drug became too common, causing treatments of infections to fail.
* Until very recently, the recommended treatment was two drugs - ceftriaxone and azithromycin.
* Azithromycin was removed from recommendations because of concern over rising resistance to the antibiotic.
* In February 2018, the first ever reported case of resistance to treatment with ceftriaxone and azithromycin, as well as resistance to the last-resort treatment spectinomycin, was reported.
* Currently in the UK, patients are only treated with ceftriaxone.

*ANTIBIOTICS*

Three antibiotics and associated unitigs are used to make a model in this problem.

*Azithromycin* :
* Azithromycin is an antibiotic used to treat various types of infections of the respiratory tract, ear, skin and eye in adults and children. It is also effective in typhoid fever and some sexually transmitted diseases like gonorrhea.

*Ciprofloxacin* :
* Ciprofloxacin is an antibiotic, used in the treatment of bacterial infections. It is also used in treating infections of the urinary tract, nose, throat, skin and soft tissues and lungs (pneumonia). It prevents the bacterial cells from dividing and repairing, thereby killing them.

*Cefixime* :
* Cefixime is an antibiotic medicine used to treat a variety of bacterial infections. It is effective in infections of the respiratory tract (eg. pneumonia), urinary tract, ear, nasal sinus, throat, and some sexually transmitted diseases.

**UNITIGS**

In our dataset, we will come across features data that will convey the presence or absence of a particular nucleotide sequence in the Bacteria's DNA

For this analysis, we're using unitigs, stretches of DNA (in string format) shared by a subset of the strains in our study.
* Unitigs are an efficient but flexible way of representing DNA variation in bacteria.
* The full dataset consists of 584,362 unitigs, which takes a long time to train models on, so for this exercise
* We will be using a set that has been filtered for unitigs associated with resistance.


## 1.2. Target Variables

*Neisseria gonorrhoeae* is either resistant (target=1) to a particular treatment or not resistant (target=0) for a particular.

AVAILABLE UNITIG DATA:

* We will be choosing one of the following antibiotic cases below:

  1.   Bacteria resistance to *Azithromycin; azm_sr*
  2.   Bacteria resistance to *Ciprofloxacin; cip_sr*
  3.   Bacteria resistance to *Cefixime; cfx_sr*

UNAVAILABLE UNITIG DATA:

* We don't currently have unitig data for the following antibiotics, so we can't check these cases:

  1.   Bacteria resistance to *Ceftriaxone; cro_sr*
  2.   Bacteria resistance to *Tetracycline; tet_sr*
  3.   Bacteria reistance to *Penicillin; pen_sr*

# 2. Dataset

## 2.1. Creating Dataset

**GET_UNITIGS CLASS**

  *   We need to do some data wrangling & combine two data files together; `.get_case`
  *   In this problem, a case type format is used (grouped feature matrix & target vector)

The feature matrix (stored within .X) will contain:

  * All collected samples from across various databses, corresponding to the bacteria's DNA (which was cut into segments of 31-mers)

  * When a particular DNA segment (unitig) is present in the bacterial sample DNA, it's value is set to 1 & 0 if it's not present

**MERGING RTAB & METADATA**

  * Filtered Unitig data is located in .Rtab files, corresponding to the same **`sample_id`** as that of the metadata

  * We need read the .Rtab files & merge the two dataframes based on the **`sample_id`** index; using .get_case()



In [14]:
''' Align Metadata Target values Unitig File & Compile Feature Matrix '''

class get_unitigs:

    def __init__(self,verbose=True):
        self.df = pd.read_csv('/content/drive/MyDrive/Dataset/metadata.csv', index_col=0) # metadata
        self.meta_names = self.df.columns
        self.target_name = None
        self.verbose = verbose

    # Get Unitig Feature matrix & Target Vector
    def get_case(self,phenotype=None):

        self.target_name = phenotype
        _metadata = self.df
        if(self.verbose):
            print(f'Target Antibiotic: {self.target_name}')
            print(f'Metadata df: {_metadata.shape}')

        # remove those that don't contain target values
        _metadata = _metadata.dropna(subset=[phenotype])
        self.metadata = _metadata.copy()

        if(self.verbose):
            print(f'Metadata df after na() removal {_metadata.shape}')
        _metadata = _metadata[phenotype] # choose target variable

        prefix = '/content/drive/MyDrive/Dataset/'
        suffix = '_gwas_filtered_unitigs.Rtab'

        if(self.verbose):
            print('\nCombining Metadata & Unitigs')

        # unitig feature matrix for phenotype
        tdf = pd.read_csv(prefix + phenotype + suffix, sep=" ",
                          index_col=0, low_memory=False)
        # align column data w/ metadata df (pattern_id = sample_idd)
        tdf = tdf.T
        # keep only common rows, ie. that have resistence measure]
        tdf = tdf[tdf.index.isin(_metadata.index)]

        train = tdf
        target = _metadata[_metadata.index.isin(tdf.index)]

        self.X = pd.concat([train,target],axis=1)
        if(self.verbose):
            print(f'Unitig Matrix (+target): {self.X.shape}')

## 2.2 Case Preview - Ciprofloxacin

  We'll be looking at bacterial resistance for three different cases, let's check one case first & load the other when required..
---

**METADATA CONTENT**

* Metadata is stored in .df, which contains our target variable (one of the _sr columns) for a specific Sample_ID

* As we can see, we don't have all the data, some data is missing in our metadata, we'll only be dropping Nan cases for the target variable (_sr) we're concerned with.

**FEATURE MATRIX**

* The feature matrix is created from one of the .Rtab files, depending on which case we are testing.

* Each column in the feature matrix is called a unitig (pattern_id); is treated as a feature for the specific Sample_ID, which is our index.

In [15]:
# Load meta data & look at the first 5 samples
case_cip = get_unitigs()

display(case_cip.df.T.iloc[:,:7])

Sample_ID,ERR1549286,ERR1549290,ERR1549291,ERR1549287,ERR1549288,ERR1549299,ERR1549292
Year,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0
Country,UK,UK,UK,UK,UK,UK,UK
Continent,Europe,Europe,Europe,Europe,Europe,Europe,Europe
Beta.lactamase,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Azithromycin,>256,>256,>256,>256,>256,>256,>256
Ciprofloxacin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ceftriaxone,0.016,0.004,0.006,0.006,0.008,0.012,0.023
Cefixime,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tetracycline,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Penicillin,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
case_cip.get_case(phenotype='cip_sr')
case_cip.X.iloc[:,:2].head()

Target Antibiotic: cip_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3088, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3088, 8874)


,ACGTTTATGCCGTTATCGATCCGATAGCCGGT,CATCTGCACCCTGTCGGCACTCGCCGCCTGAACCACCCCGTCCGGACAAGG
SRR1661154,0,0
SRR1661156,0,0
SRR1661157,0,0
SRR1661158,0,0
SRR1661159,0,0


**TARGET VARIABLE**

* The target vector, is also stored in .X, and represents the resistance property of the sample
* The Sample_ID is either 1.0 (resistant) or 0.0 (non-resistant) to a particular antibiotic in question.

In [17]:
target = case_cip.X[case_cip.target_name]
print(target[:5])

SRR1661154    0.0
SRR1661156    1.0
SRR1661157    0.0
SRR1661158    0.0
SRR1661159    0.0
Name: cip_sr, dtype: float64


## 2.3. Target Variable Distribution

**TARGET DISTRIBUTIONS & UNITIG COUNT**

* It's useful to look at the class distributions for all three cases we'll be making models for.
* As we can see below, the unitig distributions are quite different as well, so our feature matrix will vary significantly for each case.

In [18]:
''' Ciprofloxacin '''

case = get_unitigs()
case.get_case('cip_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: cip_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3088, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3088, 8874)
cip_sr
0.0    1660
1.0    1428
Name: count, dtype: int64


In [20]:
''' Azithromycin '''

case = get_unitigs()
case.get_case('azm_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: azm_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3478, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3478, 516)
azm_sr
0.0    3031
1.0     447
Name: count, dtype: int64


In [21]:
''' Cefixime '''

case = get_unitigs()
case.get_case('cfx_sr')
print(case.X[case.target_name].value_counts())

Target Antibiotic: cfx_sr
Metadata df: (3786, 30)
Metadata df after na() removal (3401, 30)

Combining Metadata & Unitigs
Unitig Matrix (+target): (3401, 385)
cfx_sr
0.0    3396
1.0       5
Name: count, dtype: int64


# 3.Feature Matrix Modification

## 3.1.Creating Case Data

* In Section 2.3, we saw that for some antibiotics, there are very few resistant samples present , mainly Cefixime
* This results in a very tricky situation if we want to use cross validation, we may not even have enough samples for the model to learn anything meaningful
* Creating models Azithromycin & Ciprofloxacin should not have this issue, as we have sufficient number of samples, even if an imbalance is present

We can try two approaches:

* Downsample the dominant class (not resistant) `.split_case`
* Utilise SMOTE based upsampling strategy `smote`

In [23]:
''' Feature Matrix Upsampling Modification (+Target) '''
# Model based approach to upsample minor class in target variable

class mod_unitigs():

    def __init__(self,unitigs):
        self.X = unitigs.X # input data class
        self.target_name = unitigs.target_name
        self.verbose = True

    ''' Downsampling Class 0 using .sample & recompile '''
    # If there's too much of the dominant class, just downsample

    def split_case(self,frac_id=0.5):

        X = self.train
        y = pd.Series(self.target,name=self.target_name)
        XX = pd.concat([X,y],axis=1)

        lst_temp = dict(tuple(XX.groupby(self.target_name))) # divide classes
        ratio = lst_temp[0].shape[0]/lst_temp[1].shape[0] # get class ratio

        # Sample approach for downsizing majority class
        X_red = lst_temp[0].sample(frac=frac_id)
        X_all = pd.concat([X_red,lst_temp[1]],axis=0)

        if(self.verbose):
            print(f'Class 0 : {lst_temp[0].shape}')
            print(f'Class 1 : {lst_temp[1].shape}')
            print(f'Class Ratio: {round(ratio,4)}')
            print(f'Reduced Training Matrix: {X_all.shape}')

        # Redefine .train, .target
        self.target = X_all[self.target_name].copy()
        X_all.drop(self.target_name, inplace=True, axis=1)
        self.train = X_all

    ''' SMOTE UPSAMPLING '''
    # For unbalanced problems, synthetically/model new data

    def smote(self,smote_id = 'smotenc',
                   smote_strat=0.5,
                   k_neighbours=5):

        self.smote_id = smote_id
        self.smote_strat = smote_strat
        self.smote_nbr = k_neighbours

        y = self.X[self.target_name].copy()
        X = self.X.drop([self.target_name],axis=1).copy()

        # smote for contin, smotenc for category
        if(self.smote_id == 'smote'):
            model_id = SMOTE(sampling_strategy=self.smote_strat,
                             k_neighbors=self.smote_nbr)
        elif(self.smote_id == 'smotenc'):
            model_id = SMOTENC(sampling_strategy=self.smote_strat,
                               k_neighbors=self.smote_nbr,
                               categorical_features=[0,1])

        X_mod, y_mod = model_id.fit_resample(X,y)
        self.X = pd.concat([X_mod,y_mod],axis=1)

        if(self.verbose):
            print(f'\nSMOTE Upsampling: {self.X.shape}')
            print(f'Target Value Counts: \n{pd.Series(y_mod).value_counts()}')
        self.X = pd.concat([X_mod,y_mod],axis=1)

# 4.Explotary Data Analysis

## 4.1. Parallel Categories

**EXPLORING THE METADATA**

* Aside from the unitig data, the Metadata Dataset contains some interesting info about each Sample_ID as well.
* Having sorted by the X_mic column, *log2_X_mic* has the same ordering as X_mic & antibiotic column name, so it was not included to reduce clutter in the figures.
* Beta.lactamase relation to antibiotic resistance doesn't seem to exhibit any particular patterns, other than Penicillin, for which the R (resistant) type almost exclsively indicates that the antibiotic will not be effective.
* Most bacteria *Sample_ID* are also shown to be of S (sensitive) type.
* If we highlight the uttermost right column, _sr, we can also note that resistance of these bacteria to antibiotics tends to be rising over time.
* These figures clearly indicate that *cefixime* by far is the most effective treatment out of the tree antibiotics.
* *Ciprofloxacin* on the otherhand has not been very effective treatment against the bacteria.

In [40]:
lst_azm = ['Year','Country','Continent','Beta.lactamase','azm_mic','Azithromycin','azm_sr']
lst_cip = ['Year','Country','Continent','Beta.lactamase','cip_mic','Ciprofloxacin','cip_sr']
lst_cfx = ['Year','Country','Continent','Beta.lactamase','cfx_mic','Cefixime','cfx_sr']
lst_antibio = [lst_azm,lst_cip,lst_cfx]

# Plot Parallel Categories Plot
def plot_pp(lst,colour='ghostwhite'):
    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    tdf['Year'] = tdf['Year'].astype(str)
    fig = px.parallel_categories(tdf)
    fig.update_traces(patch={"line": {"color":colour,'shape':'hspline'}})
    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}')
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

def plot_geomean(lst):

    global country_map

    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    # tdf['Year'] = tdf['Year'].astype(str) # Exclude 'Year' from mean calculation

    tdf['Country'] = tdf['Country'].map(country_map)
    # Select only numeric columns for mean calculation
    numeric_cols = tdf.select_dtypes(include=np.number).columns.tolist()
    tdf2 = tdf.groupby(['Country'])[numeric_cols].mean()

    fig = go.Figure(data=go.Choropleth(
        locations = tdf2.index,
        z = tdf2[lst[-1]],
        colorscale = 'magenta',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title = f'{lst[-1]}'))

    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}',
                      geo=dict(showframe=False,showcoastlines=False,
                               projection_type='equirectangular'))
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

In [28]:
for i in lst_antibio:
    plot_pp(i,'mistyrose')

## 4.2. Country Based Bacteria Resistance

**LEAST EFFECTIVE TREATMENT LOCATIONS**

The result sample pool may not be very balanced to make specific conclusions, but resistance to specific anibiotics has some geographic variation.

* For Azithromycin, samples from the US, Sweden & China are shown to be only countries with unsuccesful treatment cases.
* Globally, Ciprofloxacin has become an innefective treatment. Chile, Finland, Vietnam, China are amonst the least effective locations.
* Aside from France, Cefixime has been the most effective treatment globally.

In [38]:
lst_azm = ['Year','Country','Continent','Beta.lactamase','azm_mic','Azithromycin','azm_sr']
lst_cip = ['Year','Country','Continent','Beta.lactamase','cip_mic','Ciprofloxacin','cip_sr']
lst_cfx = ['Year','Country','Continent','Beta.lactamase','cfx_mic','Cefixime','cfx_sr']
lst_antibio = [lst_azm,lst_cip,lst_cfx]

def plot_geomean(lst):

    global country_map

    tdf = get_unitigs().df[lst]
    tdf.dropna(inplace=True)
    tdf.sort_values(by=lst[-3],inplace=True,ascending=False)
    tdf['Year'] = tdf['Year'].astype(str)

    tdf['Country'] = tdf['Country'].map(country_map)
    tdf2 = tdf.groupby(['Country']).mean()

    fig = go.Figure(data=go.Choropleth(
        locations = tdf2.index,
        z = tdf2[lst[-1]],
        colorscale = 'magenta',
        autocolorscale=False,
        reversescale=False,
        marker_line_color='black',
        marker_line_width=0.5,
        colorbar_title = f'{lst[-1]}'))

    fig.update_layout(title=f'Bacteria Resistance to {lst[-2]}',
                      geo=dict(showframe=False,showcoastlines=False,
                               projection_type='equirectangular'))
    fig.update_layout(margin=dict(t=60,b=10),height=400)
    fig.show()

In [41]:
# Plot Choropleth Map
for i in lst_antibio:
    plot_geomean(i)

## 4.3. MIC Values of All Samples

* *MIC* : a measure of the *concentration of antibiotic* bacteria can tolerate before it impairs their growth.
* We can definitely note a correlation to _mic features for antibiotic resistance in this graph alone:
  * Only small quantities of Cefixime are required to impair bacterial growth.
  * Compared to Azithromycin & Ciprofloxacin, which concentrate higher values of _mic as well
  * Hinting that it's less effective or that the bacteria is tending to become more resistant to the antibiotic & larger quantities are required to affect its function.

In [42]:
def get_mic():

    lst_cases = ['azm_mic','cip_mic','cfx_mic']
    rtabs = ['azm_sr','cip_sr','cfx_sr']
    lst_temp = []

    ii=-1
    for case in lst_cases:

        ii+=1
        case_id = get_unitigs(verbose=False)
        case_id.get_case(rtabs[ii])

        X_all = pd.concat([case_id.X,case_id.metadata],axis=1)

        new_df = X_all[case].value_counts().rename_axis(case).reset_index(name='counts')
        new_df = new_df.rename(columns={new_df.columns[0]: 'mic'})
        new_df['case'] = case
        lst_temp.append(new_df)

    X_counts = pd.concat([lst_temp[0],lst_temp[1],lst_temp[2]],axis=0)
    X_counts.sort_values(by='mic',inplace=True,ascending=True)
    X_counts['mic'] = X_counts['mic'].astype(str)

    fig = px.bar(X_counts, x='mic',y='counts',color='case')
    fig.update_layout(template='plotly_white',height=300)
    fig.show()

In [43]:
get_mic()

## 3.4. UNITIG

**UNIQUE VALUES OF ALL UNITIGS**

All columns in the feature matrix .X, used for training contains only values for *present (1)* or not *present (0)* for each Sample_ID.

In [44]:
column_values = case.X[case.target_name].values.ravel()
print(pd.unique(column_values))

[0. 1.]


**GROUPED UNITIGS**

Some columns also contain multiple *unitigs* which we can note below as well, perhaps indicating that all grouped unitigs must be present in a given Sample_ID.

In [45]:
case_unitigs = case.X.columns.tolist()

ii=-1
for i in case_unitigs:
    if(',' in i):
        ii+=1;print(f'{ii} | {i}')

0 | TTCAACTCTTCATAAGCCAAAGTCTGAATCCTCTGATC,AACAGCTCCTTAACCTGCGTGCGGTACAGGCGTTCGGCGCAGGC
1 | CTGTTTGTCTTTTTGGCGGGCGGTATGCTGA,CTGCATCCGGGGACATCCGTCGTAATCACCGCCCTGC
2 | CGCACGACGGCGTTTTGCTGCCGCTCAGCTTTGAGAAGCAGG,AAGGTTCGATCAGGAAGCCGTTGACCTTGTCGGC
3 | AAAACATCGGGGCGGTCATACCCGAATTGCGCCCCAAAGAA,TTCTCGACTGGGCAATTTTCCAGTGTCAATCCTTTGGTCTTGGTTTCCAACAGGTCTAGG
4 | GATAAGGGCAAACATCTGCGCGGCTTTGCGTCCGGAATAATAATCGCGC,CATGAAGCCGTTTACCATTGCCAAAGCATTGGATTC
5 | GCCGGTCAGCTTTGAAAAACAAGCGGTTGCGCCTAAAGGCAAGCGCGTCATCAAAGCCTCT,AATAGAGTCTGAGTTTGTTTATGTTCGGCACGGCGTTC
6 | CAATACGCCCGCCTACGATCCCAACAGACCCGGC,GAGCGCGCCGACGGAGTCCCTGTTTGCCGTACCTAAAGAGATGAAGGAAATGCCGTCTGCC,CTCGATATTTTGAAGGCGTAAACAAGGTGGTCGGCTTTT,CAATATTGTGGACAGTTTGGATTCTCCGCGCAATAAAGCCCCGCAAAACG,ATTGGATGCGGGCAAAACCGATTTGAACGAACGGCTGAATACGCAGCCT,GCGGTCATACCCGAATTGCGCCCCAAAGAAACCAGC
7 | ACTCCATGAAGAACTCAGGGAAGAACACGCCCGCCGCCCTGTCGGAA,TGCCGAACATAAACAAACTCAGACTCTATTCGATAC
8 | CTACGATCCCAACAGACCCGGCCGGGCAGACAGCGA,AATGCCGTCTGCCGCCCAATTGGAACGCCTGTCCGAGCTT,GGC

This output shows that some columns represent combined groups of multiple unitigs, and a value of 1 means that all those unitigs are present together in the sample.

**WORKING WITH SEQUENCES**

* Unitigs are composed of **nucleotides**, which means we can use the SQ() class.
* Alternatively, we can use the **BioPython** module as well, storing the sequence data in Seq instances.

In [50]:
# Using SQ() Class we can define sequences

lst_SQ = []
for unitig in case_unitigs:
    pass
# lst_SQ.append(SQ(unitig,'DNA'))

# print(type(lst_SQ[4]))

# Using BioPython we can define sequences

lst_bSQ = []
for unitig in case_unitigs:
    lst_bSQ.append(Seq(unitig))

print(type(lst_bSQ[4]))

<class 'Bio.Seq.Seq'>
